In [1]:
import bitcoin

# generator point
G = bitcoin.Point(
    x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
)

# generate the first public/private encryption key pair and the associated address
secret_key = int.from_bytes(b'Kevins secret key', 'big')
public_key = G * secret_key

public_key.address()

'mkjnvHVSu5drsYZk88Mqg198ZVdTGM68Fy'

In [2]:
# generate the second public/private encryption key pair and the associated address
secret_key2 = int.from_bytes(b'Kevins second secret key', 'big')
public_key2 = G * secret_key2

public_key2.address()

'mppjjWBZx1veQfEiZgjUdJkRwm94RHHcJg'

In [3]:
# every bitcoin transaction needs a reference to some unspent transaction output (UTXO) which is derived from a chain of ownership. In practice one would have access
# to the complete ledger, but here we are using a blockchain explorer website as our intermediary to get the UTXO information

# the public key script from the previous transaction (UTXO unlocking script)
pk_script = bitcoin.Script([
    118,
    169,
    bitcoin.hash160(public_key.encode()),
    136,
    172
])

# index and hash256 (encoded as big endian) of the UTXO
prev_idx, prev_tx = 0, bytes.fromhex('b850d8a9ec3e4a707fe61dcb4f4640c78a3b54579e755c607586ca83aefd36ff')[::-1]

In [4]:
# the goal is to send 1000 sat (10e-8 bitcoin) from public_key to public_key2
# the address associated with public_key 'mkjnvHVSu5drsYZk88Mqg198ZVdTGM68Fy' has 10,000 sat to begin with by using a testnet faucet

tx_in = bitcoin.TxIn(
    prev_tx = prev_tx,
    prev_idx = prev_idx,
    script_sig = pk_script
)

tx_out1 = bitcoin.TxOut(
    amount = 1000, # send 1000 sat to our target wallet
    script_pubkey = bitcoin.Script([
        118,
        169,
        bitcoin.hash160(public_key2.encode()),
        136,
        172
    ])
)

tx_out2 = bitcoin.TxOut(
    amount = 6500, # send 6500 sat back to origin which implies a 2500 sat tx fee
    script_pubkey = bitcoin.Script([
        118,
        169,
        bitcoin.hash160(public_key.encode()),
        136,
        172
    ])
)

# the (unsigned) transaction
tx = bitcoin.Tx(
    version = 1,
    tx_ins = [tx_in],
    tx_outs = [tx_out1, tx_out2]
)

In [5]:
# sign the transaction by using ecdsa and create a bitcoin Script which encodes this
signature = bitcoin.Signature.sign(secret_key, tx.encode())

# append 1 (SIGHASH_ALL flag) to the encoded signature, indicating this DER signature encodes all transactions
sig_bytes = signature.encode() + b'\x01'

# create the script with our encoded *unhashed* public key
script_sig = bitcoin.Script([sig_bytes, public_key.encode()])
script_sig

<Script with commands=[Element([48, 68, 2, 32, 104, 122, 42, 132, 174, 175, 56, 125, 140, 110, 151, 82, 251, 132, 72, 243, 105, 192, 245, 218, 159, 230, 149, 255, 46, 206, 183, 253, 109, 184, 183, 40, 2, 32, 84, 36, 121, 238, 208, 176, 51, 11, 42, 237, 58, 170, 208, 220, 88, 253, 171, 176, 127, 245, 157, 166, 114, 46, 198, 82, 22, 171, 139, 121, 28, 235, 1]), Element([2, 211, 34, 2, 92, 75, 156, 173, 88, 221, 134, 219, 89, 111, 112, 69, 62, 212, 89, 138, 184, 128, 162, 240, 187, 142, 11, 124, 72, 120, 37, 10, 112])]>

In [6]:
# substitute the UTXO script with our newly signed transaction
tx_in.script_sig = script_sig
tx.tx_ins = [tx_in]

# the final consolidated transaction to be broadcasted over the bitcoin network
tx.encode().hex()

'0100000001ff36fdae83ca8675605c759e57543b8ac740464fcb1de67f704a3eeca9d850b8000000006a4730440220687a2a84aeaf387d8c6e9752fb8448f369c0f5da9fe695ff2eceb7fd6db8b7280220542479eed0b0330b2aed3aaad0dc58fdabb07ff59da6722ec65216ab8b791ceb012102d322025c4b9cad58dd86db596f70453ed4598ab880a2f0bb8e0b7c4878250a70ffffffff02e8030000000000001976a914661622409325922c6f52fb47938824fedefcf8e088ac64190000000000001976a914394635d5db3be5d316367a32cbef8333e433305088ac00000000'

In [7]:
# the expected transaction id 
tx.id().hex()

'555e67457f1d906df80fc5671411baec3c38877ef281f12a1663436b0b5333c9'